In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("project.ipynb")

# Project 3: Language Models 🗣



In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
import requests
import time

In [4]:
from project import *

## Part 1: Dissecting the Corpus 🪚🫀

<a name='part1'></a>

In [11]:
def get_book(url):
    
    time.sleep(0.5)

    response = requests.get(url)
    
    book = response.text.replace('\r\n', '\n')

    start = "*** START OF THE PROJECT GUTENBERG EBOOK"
    
    start_indx = book.find(start)
    
    start_indx = book.find("***", start_indx + len(start)) + 3

    end = "*** END OF THE PROJECT GUTENBERG EBOOK"
    
    end_indx = book.find(end)

    return book[start_indx:end_indx]

In [12]:
# don't change this cell, but do run it -- it is needed for the tests
beowulf = get_book('https://www.gutenberg.org/ebooks/16328.txt.utf-8')

In [13]:
grader.check("q1")

q1 results: All test cases passed!

In [14]:
import re

def tokenize(book_string):
    # Strip leading and trailing whitespace
    book_string = book_string.strip()
    
    # Replace two or more newlines with paragraph end and start markers
    book_string = re.sub(r'\n{2,}', '\x03\x02', book_string)
    
    # Add start and end markers to the entire text
    book_string = '\x02' + book_string + '\x03'
    
    # Split the text into tokens using regex
    tokens = re.findall(r'\x02|\x03|[\w]+|[^\w\s]', book_string)
    
    return tokens


Run the cell below to prepare the autograder tests. It runs your `tokenize` function on the full body of Shakespeare's work. As a guide, you should expect the first three elements of `shakes` to be `'\x02'`, `'The'`, and `'Complete'`, and the last three elements of `shakes` to be `'William'`, `'Shakespeare'`, and `'\x03'`.

***Note***: If you are running into issues with opening the `'data/shakespeare.txt'` file, add the argument `encoding='UTF-8'` to the `open()` function.

In [15]:
# don't change this cell, but do run it -- it is needed for the tests
import time
start = time.time()
shakes = tokenize(open(Path('data') / 'shakespeare.txt').read())
elapsed = time.time() - start
elapsed # Should be (much) under 5

0.26416897773742676

In [16]:
grader.check("q2")

q2 results: All test cases passed!

In [89]:
class UniformLM(object):


    def __init__(self, tokens):

        self.mdl = self.train(tokens)
        
    def train(self, tokens):

        unique = pd.Series(tokens).unique()

        probabilities = [1/ len(unique)] * len(unique)
        
        return pd.Series(data=probabilities, index=unique)
        
    
    def probability(self, words):

        if not all(word in self.mdl for word in words):
            return 0

        probabilites = self.mdl.loc[list(words)]
        
        return float(probabilities.prod())
        
    def sample(self, M):
        
        probabilities = self.mdl.values

        tokens = self.mdl.index

        samples = np.random.choice(tokens, size=M, p=probabilities, replace=True)
        
        sentences = ' '.join(samples)
        
        return sentences

In [17]:
# don't change this cell, but do run it -- it is needed for the tests
tokens = tuple('one one two three one two four'.split())
unif = UniformLM(tokens)
unif.sample(100)

'one three two one three two two three four four four three three one three one two one four one one one four four one four four three two three two one two two two one four three four two two four two one one one two one four one two three one two one two four two three two two one three three three one two two two one two four three two one one one one two two one one two two three two two three one four two three four two three one two three one two'

In [18]:
grader.check("q3")

q3 results: All test cases passed!

In [19]:
class UnigramLM(object):
    
    def __init__(self, tokens):
        self.mdl = self.train(tokens)
    
    def train(self, tokens):
        frequency = pd.Series(tokens).value_counts()
        total = len(tokens)
        return frequency / total
    
    def probability(self, words):
        if not all(word in self.mdl for word in words): 
            return 0

        word_probabilities = self.mdl.loc[list(words)]
        return float(word_probabilities.prod())
        
    def sample(self, M):
        vocabulary = self.mdl.index
        probs = self.mdl.values
        
        selections = np.random.choice(vocabulary, size=M, p=probs, replace=True)
        sentence = ' '.join(selections)
        return sentence


In [20]:
# don't change this cell, but do run it -- it is needed for the tests
tokens = tuple('one one two three one two four'.split())
unigram = UnigramLM(tokens)
unigram.sample(100)

'two one two two one one two four one two two one one three two two two three one one one one one two three three one two one one one three one three one one two one one four four two two four two one four two three two two one one four one one two two four four one four two one two one one two one two four one one two one one one three two two one one one one one one two two two four three one one one one four two one two two'

In [21]:
grader.check("q4")

q4 results: All test cases passed!

### Conclusion: Baseline Language Models

You've now trained two baseline language models capable of generating new text from a given training text. Attempt to answer these questions for yourself before you continue.

* Which model do you think is better? Why?
* What are the ways in which both of these models are bad?

If you haven't trained your models on the `shakes` corpus, uncomment and run the cells below to do so and generate new random "sentences".

In [23]:
# Uncomment and run – should take less than 10 seconds
shakes_uniform = UniformLM(shakes)
shakes_unigram = UnigramLM(shakes)

In [24]:
# Uncomment and run
shakes_uniform.sample(10)

'houseless Love safety unshaken tinctures resolutes keen blubbering Alban chronicles'

In [25]:
# Uncomment and run
shakes_unigram.sample(10)

'for Master a do Grace this out No all bondage'

---

<a name='part3'></a>

## Part 3: Creating an N-Gram Language Model 📚

### Recap

First, let's recap what we've done so far. Recall the chain rule for probability, where $w$ is a sentence made up of tokens $w_1, w_2, ..., w_n$:

$$P(w) = P(w_1,\ldots,w_n) = P(w_1) \cdot P(w_2|w_1) \cdot P(w_3|w_1,w_2) \cdot\ldots\cdot P(w_n|w_1,\ldots,w_{n-1})$$

In Questions 3 (Uniform) and 4 (Unigram), your `train` methods computed these probabilities **unconditionally**, meaning that the probability that a token appeared in a sentence did **not depend** on the other tokens around it. That is, you said $P(w_i | w_1, w_2, ..., w_{i - 1}) = P(w_i)$. In Question 3, you let $P(w_i) = \frac{1}{\text{\# unique tokens in corpus}}$, and in Question 4, you let $P(w_i) = \frac{\text{\# tokens in corpus equal to $w_i$}}{\text{\# tokens in corpus}}$. Cruciually, each probability was determined by looking at the corpus that the model was trained on.

<br>

### N-Gram Overview

Now we will build an N-Gram language model, in which the probability of a token appearing in a sentence **does depend** on the tokens that come before it. 

The chain rule above specifies that the probability that a token occurs at in a particular position in a sentence depends on **all** previous tokens in the sentence. However, it is often the case that the likelihood that a token appears in a sentence is influenced more by **nearby** tokens. (Remember, tokens are words, punctuation, or `'\x02'` / `'\x03'`).

The N-Gram language model relies on the assumption that only nearby tokens matter. Specifically, it assumes that the probability that a token occurs depends only on the previous $N-1$ tokens, rather than all previous tokens. That is:

$$P(w_n|w_1,\ldots,w_{n-1}) = P(w_n|w_{n-(N-1)},\ldots,w_{n-1})$$

In an N-Gram language model, there is a hyperparameter that we get to choose when creating the model, $N$. For any $N$, the resulting N-Gram model looks at the previous $N-1$ tokens when computing probabilities. (Note that the unigram model you built in Question 4 is really an N-Gram model with $N=1$, since it looked at 0 previous tokens when computing probabilities.)

<br>

#### Example: Trigram Model

When $N=3$, we have a "trigram" model. Such a model looks at the previous $N-1 = 2$ tokens when computing probabilities.

Consider the tuple `('when', 'I', 'drink', 'Coke', 'I', 'smile')`, corresponding to the sentence `'when I drink Coke I smile'`. Under the trigram model, the probability of this sentence is computed as follows:

$$P(\text{when I drink Coke I smile}) = P(\text{when}) \cdot P(\text{I | when}) \cdot P(\text{drink | when I}) \cdot P(\text{Coke | I drink}) \cdot P(\text{I | drink Coke}) \cdot P(\text{smile | Coke I})$$

The trigram model doesn't consider the beginning of the sentence when computing the probability that the sentence ends in `'smile'`.

<br>

#### N-Grams

Both when working with a training corpus and when implementing the `probability` method to compute the probabilities of other sentences, you will need to work with "chunks" of $N$ tokens at a time.

**Definition:** The **N-Grams of a text** are a list of tuples containing sliding windows of length $N$.

For instance, the trigrams in the sentence `'when I drink Coke I smile'` are:

```py
[('when', 'I', 'drink'), ('I', 'drink', 'Coke'), ('drink', 'Coke', 'I'), ('Coke', 'I', 'smile')]
```

<br>

#### Computing N-Gram Probabilities

Notice in our trigram model above, we computed $P(\text{when I drink Coke I smile})$ as being the product of several conditional probabilities. These conditional probabilities are the result of **training** our N-Gram model on a training corpus.

To train an N-Gram model, we must compute a conditional probability for every $N$-token sequence in the corpus. For instance, suppose again that we are training a trigram model. Then, for every 3-token sequence $w_1, w_2, w_3$, we must compute $P(w_3 | w_1, w_2)$. To do so, we use:

$$P(w_3 | w_1, w_2) = \frac{C(w_1, w_2, w_3)}{C(w_1, w_2)}$$

where $C(w_1, w_2, w_3)$ is the number of occurrences of the trigram sequence $w_1, w_2, w_3$ in the training corpus and $C(w_1, w_2)$ is the number of occurrences of the bigram sequence  $w_1, w_2$ in the training corpus. (Technical note: the probabilities that we compute using the ratios of counts are _estimates_ of the true conditional probabilities of N-Grams in the population of corpuses from which our corpus was drawn.)

In general, for any $N$, conditional probabilities are computed by dividing the counts of N-Grams by the counts of the (N-1)-Grams they follow. 

**In the description of Question 5.2 we provide a detailed example of how we might compute such probabilities.**

<br>

### The `NGramLM` Class

The `NGramLM` class contains a few extra methods and attributes beyond those of `UniformLM` and `UnigramLM`:

1. Instantiating `NGramLM` requires both a list of tokens and a positive integer `N`, specifying the N in N-grams. This parameter is stored in an attribute `N`.
    - Assume that `N` is less than or equal to `len(tokens)`.
1. The `NGramLM` class has a method `create_ngrams` that takes in a list of tokens and returns a list of N-Grams (recall from above, an N-Gram is a **tuple** of length N). This list of N-Grams is then passed to the `train` method to train the N-Gram model.
1. While the `train` method still creates a language model (in this case, an N-Gram model) and stores it in the `mdl` attribute, this model is most naturally stored as a DataFrame. This DataFrame will have three columns:
    - `'ngram'`, containing the N-Grams found in the text.
    - `'n1gram'`, containing the (N-1)-Grams upon which the N-Grams in `ngram` are built.
    - `'prob'`, containing the probabilities of each N-Gram in `ngram`.
1. The `NGramLM` class has an attribute `prev_mdl` that stores an (N-1)-Gram language model over the same corpus (which in turn will store an (N-2)-Gram language model over the same corpus, and so on). This is necessary to compute the probability that a word occurs at the start of a text. This code is included for you in the constructor.

In [31]:
class NGramLM(object):
    
    def __init__(self, N, tokens):
        # You don't need to edit the constructor,
        # but you should understand how it works!
        
        self.N = N

        ngrams = self.create_ngrams(tokens)

        self.ngrams = ngrams
        self.mdl = self.train(ngrams)

        if N < 2:
            raise Exception('N must be greater than 1')
        elif N == 2:
            self.prev_mdl = UnigramLM(tokens)
        else:
            self.prev_mdl = NGramLM(N-1, tokens)

    def create_ngrams(self, tokens):
        n = self.N
        ngrams = []

        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            ngrams.append(ngram)
        return ngrams
        
    def train(self, ngrams):
        # N-Gram counts C(w_1, ..., w_n)
        ngram_counts = {}
        for ngram in ngrams:
            if ngram in ngram_counts:
                ngram_counts[ngram] += 1
            else:
                ngram_counts[ngram] = 1
        
        # (N-1)-Gram counts C(w_1, ..., w_(n-1))
        n_minus_one_counts = {}
        for ngram, count in ngram_counts.items():
            n_minus_one = ngram[:-1]
            if n_minus_one in n_minus_one_counts:
                n_minus_one_counts[n_minus_one] += 1
            else:
                n_minus_one_counts[n_minus_one] = 1


        # Create the conditional probabilities
        rows = []
        for ngram, count in ngram_counts.items():
            n_minus_one = ngram[:-1]
            prob = count / n_minus_one_counts[n_minus_one]  # P(w_N | prefix)
            rows.append((ngram, n_minus_one, prob))

        df = pd.DataFrame(rows, columns=['ngram', 'n1gram', 'prob'])
        return df

    def probability(self, words):
        ...
    

    def sample(self, M):
        # Use a helper function to generate sample tokens of length `length`
        ...
        
        # Transform the tokens to strings
        ...

In [137]:
class NGramLM(object):
    
    def __init__(self, N, tokens):
        self.N = N
        ngrams = self.create_ngrams(tokens)
        self.ngrams = ngrams
        self.mdl = self.train(ngrams)
        
        if N < 2:
            raise Exception('N must be greater than 1')
        elif N == 2:
            self.prev_mdl = UnigramLM(tokens)
        else:
            self.prev_mdl = NGramLM(N - 1, tokens)
    
    def create_ngrams(self, tokens):
        n = self.N
        ngrams = []
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            ngrams.append(ngram)
        return ngrams
        
    def train(self, ngrams):
        # Count each full N-gram.
        ngram_counts = {}
        for ngram in ngrams:
            ngram_counts[ngram] = ngram_counts.get(ngram, 0) + 1
        
        # Count each (N-1)-gram prefix.
        n_minus_one_counts = {}
        for ngram, count in ngram_counts.items():
            prefix = ngram[:-1]
            n_minus_one_counts[prefix] = n_minus_one_counts.get(prefix, 0) + count
        
        # Create rows with conditional probabilities: 
        # P(w_N | prefix) = count(ngram) / count(prefix)
        rows = []
        for ngram, count in ngram_counts.items():
            prefix = ngram[:-1]
            prob = count / n_minus_one_counts[prefix]
            rows.append((ngram, prefix, prob))
        df = pd.DataFrame(rows, columns=['ngram', 'n1gram', 'prob'])
        return df
    
    def probability(self, sentence):
        if self.N == 1:
            prob = 1.0
            for word in sentence:
                row = self.mdl[self.mdl['ngram'] == (word,)]
                if row.empty:
                    return 0
                prob *= row['prob'].iloc[0]
            return prob

        initial_history = sentence[:self.N - 1]
        prob = self.prev_mdl.probability(initial_history)
        
        for i in range(len(sentence) - self.N + 1):
            current_ngram = tuple(sentence[i:i+self.N])
            row = self.mdl[self.mdl['ngram'] == current_ngram]
            if not row.empty:
                prob *= row['prob'].iloc[0]
            else:
                if self.prev_mdl:
                    prob *= self.prev_mdl.probability(current_ngram[:-1])
                else:
                    return 0
        return prob

    def sample(self, M):
        tokens = ['\x02']  
        while (len(tokens) - 1) < M:
            context = tuple(tokens[-(self.N - 1):]) if len(tokens) >= (self.N - 1) else tuple(tokens)
            
            candidate_rows = self.mdl[self.mdl['n1gram'] == context]

            if candidate_rows.empty or ((len(tokens) - 1) == (M - 1)):
                tokens.append('\x03')
                break
            
            candidate_tokens = candidate_rows['ngram'].apply(lambda x: x[-1]).tolist()
            candidate_probs = candidate_rows['prob'].tolist()
            
            candidate_probs = np.array(candidate_probs, dtype=float)
            candidate_probs = candidate_probs / candidate_probs.sum()
            
            next_token = np.random.choice(candidate_tokens, p=candidate_probs)
            tokens.append(next_token)
        
        return " ".join(tokens)


In [138]:
# don't change this cell, but do run it -- it is needed for the tests
tokens = tuple('\x02 one two three one four \x03'.split())
bigrams = NGramLM(2, tokens)
out = bigrams.create_ngrams(tokens)

In [139]:
# don't change this cell, but do run it -- it is needed for the tests
tokens = "\x02 Humpty Dumpty sat on a wall , Humpty Dumpty had a great fall . \x03 \x02 All the king ' s horses and all the king ' s men couldn ' t put Humpty together again . \x03".split()
tokens = tuple(tokens)
ngram = NGramLM(2, tokens)
out_5a1 = ngram.create_ngrams(tokens)
out_5b1 = ngram.mdl
out_5c1 = ngram
out_5d1 = ngram.sample(500) 

In [140]:
out_5a1

[('\x02', 'Humpty'),
 ('Humpty', 'Dumpty'),
 ('Dumpty', 'sat'),
 ('sat', 'on'),
 ('on', 'a'),
 ('a', 'wall'),
 ('wall', ','),
 (',', 'Humpty'),
 ('Humpty', 'Dumpty'),
 ('Dumpty', 'had'),
 ('had', 'a'),
 ('a', 'great'),
 ('great', 'fall'),
 ('fall', '.'),
 ('.', '\x03'),
 ('\x03', '\x02'),
 ('\x02', 'All'),
 ('All', 'the'),
 ('the', 'king'),
 ('king', "'"),
 ("'", 's'),
 ('s', 'horses'),
 ('horses', 'and'),
 ('and', 'all'),
 ('all', 'the'),
 ('the', 'king'),
 ('king', "'"),
 ("'", 's'),
 ('s', 'men'),
 ('men', 'couldn'),
 ('couldn', "'"),
 ("'", 't'),
 ('t', 'put'),
 ('put', 'Humpty'),
 ('Humpty', 'together'),
 ('together', 'again'),
 ('again', '.'),
 ('.', '\x03')]

In [141]:
out_5b1

,ngram,n1gram,prob
0,"(, Humpty)","(,)",0.500000
1,"(Humpty, Dumpty)","(Humpty,)",0.666667
2,"(Dumpty, sat)","(Dumpty,)",0.500000
3,"(sat, on)","(sat,)",1.000000
4,"(on, a)","(on,)",1.000000
5,"(a, wall)","(a,)",0.500000
6,"(wall, ,)","(wall,)",1.000000
7,"(,, Humpty)","(,,)",1.000000
8,"(Dumpty, had)","(Dumpty,)",0.500000
9,"(had, a)","(had,)",1.000000


In [142]:
grader.check("q5")

q5 results: All test cases passed!

Now that you've built an N-Gram language model, let's use it to actually generate sentences!

Uncomment and run the cell below to define a bigram model using the `shakes` corpus and to generate a sentence of length 50 using the model. **The cell should run in under 30 seconds.**

In [143]:
# Uncomment and run
shakes_bigram = NGramLM(2, shakes)
shakes_bigram.sample(50)

'\x02 TRANIO . \x03 \x02 PETRUCHIO . I do this May show too , The son jurement de Englishman . Strong Enobarb Is the King , To me of all of riches ; By the rest you . Him I will spend in disdain , this , and let the \x03'

The hidden tests in Part 3/Question 5 will test your `NGramLM` implementation on corpuses that are much longer than the public tests. One of the corpuses we will test your implementation on is in `data/homertokens.txt`.

In [131]:
homer_tokens = tuple(open(Path('data') / 'homertokens.txt').read().split(' '))

As such, it's a good idea to make sure that you can instantiate an `NGramLM` object using `homer_tokens` and that all methods (`probability`, `sample`) run in under ~20 seconds.

In [132]:
NGramLM(5, homer_tokens)

## Congratulations, you've finished Project 3! 🎉

As a reminder, all of the work you want to submit needs to be in `project.py`.

To ensure that all of the work you want to submit is in `project.py`, we've included a script named `project-validation.py` in the project folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.

Once you've finished the project, you should open the command line and run, in the directory for this project:

```
python project-validation.py
```

**This will run all of the `grader.check` cells that you see in this notebook, but only using the code in `project.py` – that is, it doesn't look at any of the code in this notebook. If all of your `grader.check` cells pass in this notebook but not all of them pass in your command line with the above command, then you likely have code in your notebook that isn't in your `project.py`!**

You can also use `project-validation.py` to test individual questions. For instance,

```
python project-validation.py q1 q4
```

will run the `grader.check` cells for Questions 1 and 4 – again, only using the code in `project.py`.

Once `python project-validation.py` shows that you're passing all test cases, you're ready to submit your `project.py` (and only your `project.py`) to Gradescope. Once submitting to Gradescope, make sure to stick around until all test cases pass.

There is also a call to `grader.check_all()` below in _this_ notebook, but make sure to also follow the steps above.


---

To double-check your work, the cell below will rerun all of the autograder tests.

In [133]:
grader.check_all()

q1 results: All test cases passed!

q2 results: All test cases passed!

q3 results: All test cases passed!

q4 results: All test cases passed!

q5 results: All test cases passed!